<a href="https://colab.research.google.com/github/WuzI38/Data/blob/Jupyter/MidiGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Second attempt on music generation

## Download data

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/gdrive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download programgeek01/anime-music-midi

  0% 0.00/707k [00:00<?, ?B/s]
100% 707k/707k [00:00<00:00, 89.9MB/s]


In [ ]:
# Unzip data
import zipfile

zip_ref = zipfile.ZipFile("anime-music-midi.zip", "r") # Hey, don't blame me for the name, I am not this dataset's creator
zip_ref.extractall()
zip_ref.close()

In [ ]:
# enables music21 to render images of musical notes
print('installing lilypond...')
!apt-get install lilypond > /dev/null

# enables playing midi files
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null

# Copy soundfont to content directory
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

installing lilypond...
Extracting templates from packages: 100%
installing fluidsynth...


## Loading and preprocessing

### Parsing files

In [ ]:
import os
from music21 import *

path = "/content/data/undertale/"
midi_list = []
for filename in os.listdir(path):
  # print(filename)
  parsed_file = converter.parse(path + filename)
  midi_list.append(parsed_file)

In [ ]:
midi_list[0]

<music21.stream.Score 0x7f3830a259a0>

### Formatting
Possible lengths are fractions with a denominator of 12

Time signatures' denominators are always equal to 4 



In [ ]:
def fragmentise(midi_stream):
  return [part.flat.notes for part in midi_stream.parts]

In [ ]:
def signature(midi_stream):
  sig = midi_stream.getTimeSignatures()[0]
  return [sig.numerator, sig.denominator]

In [ ]:
frag = fragmentise(midi_list[15])
frag

[<music21.stream.iterator.StreamIterator for Part:0x7f383146a8e0_flat @:0>,
 <music21.stream.iterator.StreamIterator for Part:0x7f38310752b0_flat @:0>]

In [ ]:
sg = signature(frag[0])
sg

[4, 4]

In [ ]:
Piece is separated into 

In [ ]:
def part_formatter(midi_part):
  # Part one 

In [ ]:
# (This cell is mostly stolen from kaggle, but does it really matter?)
# I turned it into a dict cuz I have no idea how to use the library

def part_to_dict(midi_part):
  # If what I think is correct midi stream is divided into midi parts which 
  # Represents instruments (or left/right hand piano melody)
  notes_as_dict = []
  for n in midi_part.flat.notes:     
    sound_dict = dict()
    if isinstance(n, note.Note):
      sound_dict["type"] = "Note"
      sound_dict["pitch"] = max(0.0, n.pitch.ps)
      sound_dict["duration"] = n.duration.quarterLength
    elif isinstance(n, chord.Chord):
      sound_dict["type"] = "Chord"
      sound_dict["pitches"] = [] 
      for pitch in n.pitches:
        sound_dict["pitches"].append(max(0.0, pitch.ps))
    sound_dict["duration"] = n.duration.quarterLength
    sound_dict["offset"] = n.offset
    notes_as_dict.append(sound_dict)
    
  return notes_as_dict

part_to_dict(frag[1])[:50]

[{'type': 'Note', 'pitch': 38.0, 'duration': 0.25, 'offset': 3.5},
 {'type': 'Note', 'pitch': 43.0, 'duration': 0.5, 'offset': 4.0},
 {'type': 'Note', 'pitch': 38.0, 'duration': Fraction(1, 3), 'offset': 5.0},
 {'type': 'Note', 'pitch': 43.0, 'duration': Fraction(1, 3), 'offset': 6.0},
 {'type': 'Note', 'pitch': 38.0, 'duration': Fraction(1, 3), 'offset': 7.0},
 {'type': 'Note', 'pitch': 43.0, 'duration': 0.5, 'offset': 8.0},
 {'type': 'Note', 'pitch': 38.0, 'duration': Fraction(1, 3), 'offset': 9.0},
 {'type': 'Note', 'pitch': 43.0, 'duration': Fraction(1, 3), 'offset': 10.0},
 {'type': 'Note', 'pitch': 38.0, 'duration': Fraction(1, 3), 'offset': 11.0},
 {'type': 'Note', 'pitch': 43.0, 'duration': 0.5, 'offset': 12.0},
 {'type': 'Note', 'pitch': 38.0, 'duration': Fraction(1, 3), 'offset': 13.0},
 {'type': 'Note', 'pitch': 43.0, 'duration': Fraction(1, 3), 'offset': 14.0},
 {'type': 'Note', 'pitch': 38.0, 'duration': 0.25, 'offset': 15.0},
 {'type': 'Note', 'pitch': 38.0, 'duration': 0

In [ ]:
durs = set()
for lst in midi_list:
  for frg in fragmentise(lst):
    durs.add(signature(frg)[1])

In [ ]:
durs

{4}